# Results

In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)

import pandas as pd
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
matplotlib.rcParams["figure.figsize"] = (15,4)

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
only_joined = global_variables[
    "only_joined"
]  # (regress/predict with only models from joined runs)S

In [ ]:
regressions = global_variables['regressions']

regression_names = ['motion_regression','force_regression']

def load_joined(join='joined'):
    df_accuracies = pd.DataFrame()
    
    for vmm_name in vmms:
        for model_test_id in model_test_ids:
            
            file_paths = []
            file_paths.append(f"{ vmm_name }.motion_regression.{join}.{model_test_id}.simulation_accuracy")
            file_paths.append(f"{ vmm_name }.force_regression.{model_test_id}.simulation_accuracy")
            
            
            for regression,file_path in zip(regression_names, file_paths):
                try:
                    accuracy = catalog.load(file_path)
                except:
                    continue
                
                accuracy = pd.Series(accuracy)
                accuracy['total'] = accuracy.mean()
                
                accuracy['vmm'] = vmm_name
                accuracy['regression'] = regression
                accuracy['model test'] = model_test_id
                
                df_accuracies = df_accuracies.append(accuracy, ignore_index=True)
                
    return df_accuracies
            

In [ ]:
df_accuracies = load_joined()

In [ ]:
df_compare = df_accuracies.groupby(by=['vmm','regression']).mean().sort_values(by='total', ascending=True)
df_compare

In [ ]:
df_compare.reset_index().pivot(index=['vmm'], columns=['regression'], values='total').plot.bar();